# Gemini API Masterclass

In [1]:
import getpass
import os
from google import genai

# Securely prompt for the API key
if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Gemini API key: ")

# Initialize the client
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

Enter your Gemini API key:  ········


## Basic Text Generation
This is the "Hello World" of Gemini. We'll use the Gemini 2.0 Flash model, which is the current standard for speed and efficiency.

In [2]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Explain how an LLM works using a baking analogy."
)

print(f"Gemini's Response:\n{'-'*20}")
print(response.text)

Gemini's Response:
--------------------
Okay, let's bake an LLM! Imagine you want to create a delicious cake. An LLM is like a *super-powered, recipe-generating baker* that can come up with variations and continuations of cake recipes based on what it's already seen.

Here's the breakdown:

**1. The Data: The Cookbook (Training Data)**

*   **What it is:**  The core of an LLM is a massive dataset of text. Think of this as a **giant cookbook filled with countless cake recipes.** These recipes range from simple sponge cakes to elaborate wedding cakes. They include ingredients, instructions, reviews, and even stories about cake baking.
*   **Analogy:** The more diverse and well-written the recipes in your cookbook, the better your LLM baker will learn. You need recipes in different styles, with varying levels of detail, and covering a wide range of cake types.

**2. Tokenization: Breaking Down the Ingredients & Instructions (Tokenizing)**

*   **What it is:** Before the baker can use the 

## Multimodal Capabilities (Images)
Gemini is natively multimodal. You can show the audience how to "see" an image by passing a URL or a local file.

In [3]:
from IPython.display import Image, display

# Example: Analyzing an image from a URL
image_url = "https://storage.googleapis.com/generativeai-downloads/images/composition.png"
display(Image(url=image_url, width=400))

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        "Describe what is happening in this image and identify the main objects.",
        image_url # The SDK handles the URL directly!
    ]
)

print(response.text)

Here's a description of the image and identification of the main objects:

**Description:**

The image appears to be a still life or a staged composition. It features a selection of everyday objects arranged on what looks like a simple surface, possibly a table. The lighting is soft and diffused, creating a gentle, almost painterly aesthetic. There's a sense of quiet contemplation and the deliberate arrangement of the items suggests a focus on form, texture, and the relationships between them.

**Main Objects:**

*   **Earthenware objects:** An earthenware vase, possibly terracotta in color, that could be the focal point of the arrangement.
*   **Fruits:** A ripe pear or peach, its rounded form providing a contrast to the angular lines of other objects.

This composition of simple and natural forms suggests that the artist or photographer is interested in the beauty of the mundane and the visual harmony that can be found in everyday life.


## Setting a "System Persona"
System instructions allow you to define the model's behavior, tone, and constraints throughout the entire interaction.

In [5]:
from google.genai import types

# Define the "rules" for the AI
sys_instruct = "You are a grumpy but helpful Senior Python Architect. Keep answers concise and use technical jargon."

# Pass the instruction into the config
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="How do I fix a RecursionError?",
    config=types.GenerateContentConfig(system_instruction=sys_instruct)
)

print(response.text)

`RecursionError` in Python signals that a recursive function has exceeded the maximum recursion depth. Debug by:

1.  **Base Case:** Ensure your recursive function has a well-defined base case to terminate recursion.
2.  **Progress:** Verify that each recursive call moves closer to the base case. Missing or incorrect progress can lead to infinite recursion.
3.  **Tail Call Optimization:** Python lacks TCO; refactor to iterative solutions where feasible.
4.  **Increase Recursion Limit (Caution):** `sys.setrecursionlimit(limit)` can increase the limit, but this masks underlying issues and can cause stack overflow.

Example:

```python
import sys

def recursive_function(n):
    if n == 0:  # Base case
        return 0
    else:
        return n + recursive_function(n - 1)  # Progress towards base case

# sys.setrecursionlimit(1500) # use with caution
print(recursive_function(100))
```

If the root cause is complex data structures (e.g., deeply nested trees), consider restructuring the dat

## Multi-turn Chat (Stateful)
In a standard API call, the model doesn't "remember" previous messages. Using chats.create handles the history for you automatically.

In [6]:
# Start a chat session
chat = client.chats.create(model="gemini-2.0-flash")

# Turn 1
response1 = chat.send_message("Hi, I'm building a robot named 'Sparky'.")
print(f"Gemini: {response1.text}\n")

# Turn 2 (Gemini remembers the name 'Sparky')
response2 = chat.send_message("What's a good first task for my robot?")
print(f"Gemini: {response2.text}")

# You can view the history at any time:
# print(chat.get_history())

Gemini: That's awesome! Building a robot is a fantastic project. Tell me more about Sparky! I'm excited to hear about it. To help me understand and offer helpful advice, could you tell me about:

*   **What is Sparky supposed to do?** (What is its purpose? Is it for fun, a specific task, learning, etc.?)
*   **What are its main features?** (e.g., wheels, arms, sensors, etc.)
*   **What platform are you using?** (e.g., Arduino, Raspberry Pi, custom circuit, etc.)
*   **What programming language are you using?** (e.g., Python, C++, etc.)
*   **What is your current skill level in robotics?** (Beginner, intermediate, advanced)
*   **What challenges are you facing right now?** (If any)
*   **What are your goals for Sparky?** (short-term and long-term)

The more details you provide, the better I can assist you. Good luck with Sparky!


Gemini: A great first task for a robot like Sparky should be something simple, achievable, and that allows you to test core functionalities. Here are a few su

## Function Calling (The "Agentic" Example)

In [8]:
# 1. Define a tool (a real Python function)
def get_weather(location: str):
    """Returns the weather for a given location."""
    # In a real app, you'd call a weather API here
    return f"The weather in {location} is 22°C and sunny."

# 2. Tell Gemini about the tool
# 'automatic_function_calling' means the SDK handles the execution for you!
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        tools=[get_weather]
    )
)

# 3. Ask a question that requires the tool
response = chat.send_message("what is the weather of the London?")
print(response.text)

The weather in London is 22°C and sunny.

